In [56]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [57]:
# 前置知识
features = np.random.normal(size=(5 + 5, 1)) #正太分布生成随机数

poly_features = np.power(features, np.arange(5).reshape(1, -1)) # 输入的x随机

features,poly_features

(array([[ 1.64254566],
        [ 0.2236447 ],
        [ 1.43055395],
        [-1.42776258],
        [ 0.37949699],
        [ 0.25899349],
        [ 0.511566  ],
        [ 0.63209722],
        [ 0.01263846],
        [-1.21458633]]),
 array([[ 1.00000000e+00,  1.64254566e+00,  2.69795624e+00,
          4.43151630e+00,  7.27896785e+00],
        [ 1.00000000e+00,  2.23644696e-01,  5.00169502e-02,
          1.11860256e-02,  2.50169531e-03],
        [ 1.00000000e+00,  1.43055395e+00,  2.04648460e+00,
          2.92760662e+00,  4.18809921e+00],
        [ 1.00000000e+00, -1.42776258e+00,  2.03850598e+00,
         -2.91050256e+00,  4.15550664e+00],
        [ 1.00000000e+00,  3.79496995e-01,  1.44017969e-01,
          5.46543865e-02,  2.07411754e-02],
        [ 1.00000000e+00,  2.58993490e-01,  6.70776278e-02,
          1.73726689e-02,  4.49940815e-03],
        [ 1.00000000e+00,  5.11565997e-01,  2.61699769e-01,
          1.33876703e-01,  6.84867692e-02],
        [ 1.00000000e+00,  6.32097215e-0

In [58]:
max_degree = 20  # 多项式的最大阶数  其实就是 x最高幂是多少
n_train, n_test = 100, 100  # 训练和测试数据集大小
true_w = np.zeros(max_degree)  # 分配大量的空间
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])

features = np.random.normal(size=(n_train + n_test, 1)) #正太分布
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1)) #得到的就是x^0 + x^1+..+x^max_degree-1     总共训练集和测试集200个样本
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  # gamma(n)=(n-1)!
# labels的维度:(n_train+n_test,)
labels = np.dot(poly_features, true_w) # 200*20 × 20*1 得到真正的labels
labels += np.random.normal(scale=0.1, size=labels.shape) #加一点噪音，等会拟合的时候上点难度
# true_w,features,poly_features
features,poly_features

(array([[ 0.01442141],
        [ 1.66603734],
        [-0.74595713],
        [ 0.99011266],
        [ 0.29457451],
        [ 0.46402243],
        [-1.86307662],
        [ 0.10044282],
        [ 1.34571665],
        [-0.21783803],
        [-0.96195143],
        [ 0.27073849],
        [ 1.97510631],
        [ 0.29545702],
        [ 0.40010602],
        [ 2.14107738],
        [ 0.19171322],
        [ 0.23459035],
        [-0.83327544],
        [ 0.64823983],
        [ 0.48277579],
        [ 0.69907124],
        [-1.04688766],
        [-1.74020823],
        [-1.04853385],
        [ 1.59010427],
        [-0.07377866],
        [-0.37415038],
        [ 0.06024873],
        [ 0.459935  ],
        [-0.18112082],
        [ 2.22953111],
        [-0.39261694],
        [-0.70978883],
        [-0.02641785],
        [ 1.45920878],
        [-0.59678397],
        [ 0.40412674],
        [ 1.40994172],
        [-0.58754083],
        [ 0.04515519],
        [-0.31866796],
        [-1.2450711 ],
        [ 2

In [59]:
features[:2], poly_features[:2, :], labels[:2]

(array([[0.01442141],
        [1.66603734]]),
 array([[1.00000000e+00, 1.44214084e-02, 1.03988510e-04, 4.99886923e-07,
         1.80226836e-09, 5.19824962e-12, 1.24943468e-14, 2.57408682e-17,
         4.64024465e-20, 7.43542923e-23, 1.07229361e-25, 1.40581674e-28,
         1.68948811e-31, 1.87421523e-34, 1.93063023e-37, 1.85616046e-40,
         1.67302800e-43, 1.41926001e-46, 1.13709601e-49, 8.63080310e-53],
        [1.00000000e+00, 1.66603734e+00, 1.38784020e+00, 7.70731197e-01,
         3.21016737e-01, 1.06965174e-01, 2.97013289e-02, 7.06907469e-03,
         1.47216780e-03, 2.72520723e-04, 4.54029700e-05, 6.87664029e-06,
         9.54728288e-07, 1.22354844e-07, 1.45605527e-08, 1.61722830e-09,
         1.68397670e-10, 1.65033415e-11, 1.52751018e-12, 1.33941525e-13]]),
 array([4.8926845 , 6.51445069]))

In [60]:
def evaluate_loss(net, data_iter, loss):
    """评估给定数据集上模型的损失"""
    metric = d2l.Accumulator(2)  # 损失的总和,样本数量
    for X, y in data_iter:
        out = net(X)
        y = y.reshape(out.shape)
        l = loss(out, y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1] #正确预测数/预测总数 = 预测准确率

In [61]:
def train(train_features, test_features, train_labels, test_labels,
          num_epochs=400):
    loss = nn.MSELoss(reduction='none') # Mean Square Error Loss
    input_shape = train_features.shape[-1]
    # 不设置偏置，因为我们已经在多项式中实现了它
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels.reshape(-1,1)), batch_size=batch_size)
    test_iter = d2l.load_array((test_features, test_labels.reshape(-1,1)), batch_size=batch_size, is_train=False)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log',
                            xlim=[1, num_epochs], ylim=[1e-3, 1e2],
                            legend=['train', 'test']) # 可视化损失值变化
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss),
                                     evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data.numpy())

In [62]:
# 从多项式特征中选择前4个维度，即1,x,x^2/2!,x^3/3!
train(poly_features[:n_train, :4], poly_features[n_train:, :4],labels[:n_train], labels[n_train:])

TypeError: 'int' object is not callable

不知道为啥运行不了。。。